# 卷积层

一张 12,000,000 像素的猫或者狗的图片，分RGB三种颜色，如果使用单隐藏层的 MLP 处理，模型有 3.6B 个参数，远多于世界上所有猫和狗的数量。

## 1. 识别器的原则

- 平移不变性（在图片的哪个位置都能识别到）
- 局部性（不需要看整张图就能识别）

## 2. 重新考察全连接层

- 将输入和输出变形为矩阵（宽度，高度）
- 将权重变形为4-D张量 $(h, w)$ 到 $(h^\prime, w^\prime)$


## 3. 原则 #1 - 平移不变性

- x的平移导致h的平移
- v不应该依赖于(i, j)
- 解决方案：

## 4. 原则 #2 - 局部性

$h_{i,j} = \sum_{a,b} \mathbf{v}_{i,j,a,b} \mathbf{x}_{i+a,j+b}$

## 5. 小结

- 对全连接层使用平移不变性和局部性得到卷积层

$~~~~~~~~~~~h_{i,j} = \sum_{a,b} \mathbf{v}_{i,j,a,b} \mathbf{x}_{i+a,j+b}$

$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$⬇

$~~~~~~~~~~~h_{i,j} = \sum_{a=-\Delta}^{\Delta} \sum_{b=-\Delta}^{\Delta} \mathbf{v}_{a,b} \mathbf{x}_{i+a,j+b}$

## 6. 二维交叉相关

- 卷积核 (Kernel)，其实就是 $\mathbf{w}$
- 核是不随扫描的位置变化的，这就实现了平移不变性

## 7. 二维卷积层

- 输入 $\mathbf{X}: n_h \times n_w$
- 核 $\mathbf{W}: k_h \times k_w$
- 偏差 $b \in \mathbb{R}$
- 输出 $\mathbf{Y}: (n_h - k_h + 1) \times (n_w - k_w + 1)$

$\mathbf{Y} = \mathbf{X} \mathbf{W} + b$

$\mathbf{W}$ 和 $b$ 是可学习的参数

## 8. 例子

不同的卷积核，会得到不同的效果

边缘检测

$\left[\begin{array}{c}
-1 & -1 & -1 \\
-1 & 8 & -1 \\
-1 & -1 & -1
\end{array}\right]$

锐化

$\left[\begin{array}{c}
0 & -1 & 0 \\
-1 & 5 & -1 \\
0 & -1 & 0
\end{array}\right]$

高斯模糊

$\frac{1}{16}\left[\begin{array}{c}
1 & 2 & 1 \\
2 & 4 & 2 \\
1 & 2 & 1
\end{array}\right]$

## 9. 交叉相关 vs 卷积

- 二维交叉相关
- 二维卷积
- 由于对称性，在实际使用中没有区别

因为使用是一样的，所以没有使用数学上严格定义的卷积

## 10. 一维和三维交叉相关

- 一维：文本、语言、时需序列
- 二维：视频、医学图像、气象图像

## 11. 总结

- 卷积层将输入和核矩阵进行交叉相关，加上偏移后得到输出
- 核矩阵和偏移是可学习的参数
- 核矩阵的大小是超参数


## 12. 互相关运算

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y

验证上述二维互相关运算的输出

In [2]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## 13. 实现二维卷积层

In [3]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torcj.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

## 14. 卷积层的一个简单应用：检测图像中不同颜色的边缘

In [4]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [8]:
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

卷积核K只可以检测垂直边缘，不能检测横向边

In [10]:
X.t()

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

In [11]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 15. 学习由 `X` 生成 `Y` 的卷积核

In [13]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum():.3f}')

batch 2, loss 6.775
batch 4, loss 2.265
batch 6, loss 0.842
batch 8, loss 0.331
batch 10, loss 0.133


In [19]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9550, -1.0299]])